# Task 2.1: LSH from scratch

In [12]:
!pip install mmh3

  Using cached https://files.pythonhosted.org/packages/fa/7e/3ddcab0a9fcea034212c02eb411433db9330e34d626360b97333368b4052/mmh3-2.5.1.tar.gz
  Running setup.py bdist_wheel for mmh3 ... error
  Complete output from command /Users/alexander/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/2k/shj0p2n928j7fjq528nlws040000gn/T/pip-install-cqewgyjx/mmh3/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/2k/shj0p2n928j7fjq528nlws040000gn/T/pip-wheel-z2xaa6eu --python-tag cp36:
  running bdist_wheel
  running build
  running build_ext
  building 'mmh3' extension
  creating build
  creating build/temp.macosx-10.7-x86_64-3.6
  gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/alexander/anaconda3/include -arch x86_64 -I/Users/alexander/anaconda3/include -arch x86_64 -I/User

In [41]:
from mmh3 import hash  # murmur hash 
from random import randint

import numpy as np

## Зададим игрушечный корпус документов

In [42]:
corpus = [
          "Who was the first king of Poland",
          "Who was the first ruler of Poland",
          "Who was the last pharaoh of Egypt", 
        ]

Опишем функцию шинглирования корпуса.

Шинглирование может происходит по словам и по словесным нграммам, также добавим возможность хешировать на лету слова/нграммы. Это позволит работать со словами вне словаря и сократить размер признакового пространства. 

In [45]:
def shingling(corpus, k=0, hash_shingles=False, buckets=2**15):
    # corpus = list of strings
    # k - size of char-ngrams for shingling
    # hash_shingles - use shingle hashing or not
    # buckets - num of buckets for hashing if hash_shingles=True
    
    if k:
        shingled_corpus = [[phrase[i:i+k] for i in range(len(phrase) - k+1) ] for phrase in corpus]
    
    else: #if k = 0 - use words as shingles
        shingled_corpus = [phrase.split() for phrase in corpus]
    
    if hash_shingles: 
        shingled_corpus = [[mmh.hash(shingle, signed=False) % buckets for shingle in doc] for doc in shingled_corpus]
    
    return shingled_corpus
    

In [46]:
shingled_corpus = shingling(corpus)

In [47]:
shingled_corpus

[['Who', 'was', 'the', 'first', 'king', 'of', 'Poland'],
 ['Who', 'was', 'the', 'first', 'ruler', 'of', 'Poland'],
 ['Who', 'was', 'the', 'last', 'pharaoh', 'of', 'Egypt']]

Определим функцию, считающую коэффициент Жаккара между документами

In [51]:
def jaccard_sim(l1, l2):
    # l1, l2 - lists of tokens
    s1 = set(l1)
    s2 = set(l2)
    
    intersection_size = len(s1.intersection(s2))
    union_size = len(s1.union(s2))
    
    return intersection_size/union_size
    

In [52]:
doc1, doc2 = shingled_corpus[:2]
jaccard_sim(doc1, doc2)  # как и следовало ожидать

0.75

###  Зададим функцию, генерирующую минхэш-сигнатуру

In [53]:
def generate_minhash_signature(doc, num_perm=100):
    # doc - list of shinles
    # num_perm - number of hash functions to use
    
    minhash_signature = []

    for i in range(num_perm):
        new_hash = lambda x: mmh.hash(str(x), signed=False) % (100000+i)  # хэш-функции должны быть разными
        minhash = min([new_hash(shingle) for shingle in doc])  # take only minimum of hashed values
        minhash_signature.append(minhash)
        
    return minhash_signature

Попробуем сгенерировать минхэш-сигнатуру для первого и второго документа в коллекции

In [54]:
mhash_size = 12  # 12 hash functions to use
 
minhash_doc0 = generate_minhash_signature(shingled_corpus[0], mhash_size)  
minhash_doc1 = generate_minhash_signature(shingled_corpus[1], mhash_size)

print(minhash_doc0)
print(minhash_doc1)

NameError: name 'mmh' is not defined

Заметим, что здесь считать коэффициент Жаккара нужно, сравнивая элементы попарно, а не мешая их в одно множество

In [55]:
def jaccard_sim_mhash(l1, l2):
    # l1, l2 - lists of minhashes
    l1 = np.array(l1)
    l2 = np.array(l2)
    
    intersection = (l1 == l2).sum()
    
    return intersection/l1.size
        

In [56]:
jaccard_sim_mhash(minhash_doc0, minhash_doc1)

NameError: name 'minhash_doc0' is not defined

#### Примерно оценить размер группы и количество групп можно, исходя из того порога коэффициента Жаккара, который мы зададим для "похожих" документов

Будем считать "похожими" документы с коэффициентов Жаккара, большим 0.7

In [57]:
mhash_size = 12
n = 4
b = mhash_size/n #  количество групп
b

3.0

При выбранных $n$ и $b$ порог должен быть примерно равен ${1\over b}^{1\over n}$

In [58]:
(1/b)**(1/n) #  больше 0.7 - значит группы, размера 4 подойдут

0.7598356856515925

### Теперь опишем функцию, разбивающую сигнатуры по группам 

In [59]:
def get_supershingles(signature, n=4):
    # signature - list of minhashes
    return [signature[i:i+n] for i in range(0,len(signature),n)]


In [60]:
print(minhash_doc0)
print(get_supershingles(minhash_doc0))

NameError: name 'minhash_doc0' is not defined

## Теперь каждая группа - ключ, по которому можно искать похожие документы в хеш-таблице

При наличии хотя бы одной совпадающей группы минхэшей в запросе и обработанном корпусе, запрос и соответствующий документ считаются потенциально похожей парой документов

Давайте заведем такую хеш-таблицу, которая будет хранить информацию о корпусе.
Заметим, что строго говоря, группа минхешей **неинвариантна относительно местонахождения**


Проще на примере:
- l1 =  (  [2,1,1]   [3,3,4]   )
- l2 =  (  [1,2,3]   [2,1,1]   )

Группа [2,1,1] есть в обоих сигнатурах, но помним, что сранивать сигнатуры в данном случае нужно попарно между собой. Таким образом общая группа в одном и в другом случаях отражает разные преобразования входных данных.

Попарное сравнение - дает коэффициент Жаккара = 0

Так что будем при формировании хеш-таблицы учитывать порядковый номер следования группы

In [61]:
shingled_corpus

[['Who', 'was', 'the', 'first', 'king', 'of', 'Poland'],
 ['Who', 'was', 'the', 'first', 'ruler', 'of', 'Poland'],
 ['Who', 'was', 'the', 'last', 'pharaoh', 'of', 'Egypt']]

In [62]:
# Параметры
params = {'mhash_size': 12, "n":4, "k_shingles":0, "hash_shingles":False, "buckets":2**15}

In [63]:
def lsh_training(corpus, parameters):
    """shingled_corpus - list of lists with documents' shingles
    
    
    """
    lsh_dict = {}
    
    shingled_corpus = shingling(corpus, parameters["k_shingles"], 
                                parameters["hash_shingles"], 
                                parameters["buckets"])
    
    for doc_id, doc in enumerate(shingled_corpus):
        minhash_signature = generate_minhash_signature(doc, num_perm=parameters["mhash_size"])
        minhash_groups = get_supershingles(minhash_signature, n=parameters["n"])
        
        for i,group in enumerate(minhash_groups):
            hash_key = hash(str(group),signed=False) % (100000+i)   # превращаем список с группой в строку, хешируем с учетом порядекового номера группы в сигнатуре 
            lsh_dict[hash_key] = corpus[doc_id]
    
    return lsh_dict

In [64]:
lsh_words = lsh_training(corpus, params)

NameError: name 'mmh' is not defined

In [65]:
lsh_words

NameError: name 'lsh_words' is not defined

------------------------------

## Допустим теперь есть запрос и мы хотим найти максимально похожие документы в базе

Для запроса нужно провести все те же операции, чято и для документов в базе. Заметим, что набор хэш-функций **обязан остаться тем же самым**. Иначе все просто бессмысленно

In [66]:
def find_similar(query, lsh_dict, parameters):
    # query - document as string
    possible_duplicates = set()
    query = [query]
    shingled_query = shingling(query, parameters["k_shingles"], 
                                parameters["hash_shingles"], 
                                parameters["buckets"])  # параметры шинглирования должны быть те же, что и для корпуса
    
    query_signature = generate_minhash_signature(shingled_query[0], parameters["mhash_size"])
    query_groups = get_supershingles(query_signature, n=parameters["n"])
    
    for i,group in enumerate(query_groups):
        hash_key = hash(str(group),signed=False) % (100000+i)
        if lsh_dict.get(hash_key):
            possible_duplicates.add(lsh_dict[hash_key])

    if possible_duplicates:
        print("Possible duplicates for query:")
        for doc in possible_duplicates:
            print(doc)
            
    else:
        print("Duplicates for query not founded")

In [67]:
find_similar("Who was the first king of", lsh_words, params)

NameError: name 'lsh_words' is not defined

In [68]:
find_similar("Who was the first king of Polan", lsh_words, params)

NameError: name 'lsh_words' is not defined

In [69]:
find_similar("Who was the last", lsh_words, params)

NameError: name 'lsh_words' is not defined

## Шинглирование по нграммам помогает бороться с ошибками и опечатками в запросе

In [70]:
# Параметры
params_ngrams = {'mhash_size': 12, "n":3, "k_shingles":5, "hash_shingles":False, "buckets":2**15}

In [71]:
lsh_ngrams = lsh_training(corpus, params_ngrams)

NameError: name 'mmh' is not defined

In [72]:
find_similar("Who was the first king of Polan", lsh_ngrams, params_ngrams)

NameError: name 'lsh_ngrams' is not defined

In [73]:
find_similar("Who was the first king of Poand", lsh_ngrams, params_ngrams)

NameError: name 'lsh_ngrams' is not defined

In [74]:
find_similar("the first king of Poland", lsh_ngrams, params_ngrams)

NameError: name 'lsh_ngrams' is not defined

## SimHash

Simhash - алгоритм, альтернативный минхэшу - превратить последовательность шинглов в сигнатуру (только сигнатура получатется немного по другому). 

После получения сигнатуры выполняется LSH: разбиение сигнатуры на супершинглы, строится таблица дубликатов

Основная идея состоит в том, что для каждого шингла формируются n-битные бинарные хеши, которые затем особым образом складываются

## Функция, получающая от шингла n_bit-ный бинарный хеш

In [75]:
def get_binary_hash(s, n_bit=8):
    binary_hash = ("{0:0%sb}"%str(n_bit)).format(hash(s, signed=False)%(2**n_bit-1))
    return binary_hash

In [76]:
get_binary_hash("Hello!")

TypeError: 'signed' is an invalid keyword argument for this function

## Функция, получающая из документа SimHash сигнатуру

- получаем для шинглов обычный 32-битный integer хеш
- хешируем его на **$2^{n\_bit}-1$** bucket'ов, чтобы получить n_bit-ный бинарный хеш  
- затем в бинарных хешах превращаем все 0 в -1
- затем взвешиваем бинарные хеши для каждого шингла в соответствии с term-frequency для данного шингла (опционально)
- затем побитово суммируем все хеши для каждого шингла
- затем бинаризуем побитово суммарный хеш: если компонент > 0 - ставим 1, иначе - ставим 0
- полученная сигнатура и есть SimHash сигнатура документа 

In [77]:
def get_simhash(doc, n_bit=8):
    # doc - list of shingles    
    hashes = [get_binary_hash(shingle, n_bit=n_bit) for shingle in doc] # get binary hashes for all shingles
    hashes = np.array([[1 if bit == "1" else -1 for bit in bithash] for bithash in hashes])   # map 0 to -1 
    simhash = np.array([1 if i>0 else 0 for i in hashes.sum(axis=0)]) # sum up all hashes bitwise and binarize final hash

    return simhash

In [78]:
doc = corpus[0].split()

In [79]:
simhash_signature = get_simhash(doc)
simhash_signature

TypeError: 'signed' is an invalid keyword argument for this function

In [80]:
get_supershingles(simhash_signature, n=4)

NameError: name 'simhash_signature' is not defined

###  Функция близости между векторами:

In [81]:
def get_similarity(simhash1, simhash2):
    # FYI: Hamming distance for bit vectors = number of different components for pair of vectors
    # Hamming([1,0,1,0,1], [1,0,1,0,0]) = 1 - only last bit differs
    return (simhash1 == simhash2).sum() / len(simhash1)

Какое сходство между первыми двумя документами корпуса?

In [82]:
doc1 = corpus[0].split()
simhash_signature1 = get_simhash(doc1)

doc2 = corpus[1].split()
simhash_signature2 = get_simhash(doc2)

get_similarity(simhash_signature1, simhash_signature2)

TypeError: 'signed' is an invalid keyword argument for this function

Они очень похожи с точки зрения Симхэша, даже более, чем с точки зрения Минхэш сигнатуры

## Давайте сделаем LSH с SimHash сигнатурой и посмотрим, что получится

In [83]:
# Параметры
params = {"n_bits": 16, "n":4, "k_shingles":0, "hash_shingles":False, "buckets":2**15}

In [84]:
def lsh_training(corpus, parameters):
    """shingled_corpus - list of lists with documents' shingles
    
    
    """
    lsh_dict = {}
    
    shingled_corpus = shingling(corpus, parameters["k_shingles"], 
                                parameters["hash_shingles"], 
                                parameters["buckets"])
    
    for doc_id, doc in enumerate(shingled_corpus):
        simhash_signature = get_simhash(doc, n_bit=parameters["n_bits"])
        simhash_groups = get_supershingles(simhash_signature, n=parameters["n"])
        
        for i,group in enumerate(simhash_groups):
            hash_key = hash(str(group),signed=False) % (100000+i)   # превращаем список с группой в строку, хешируем с учетом порядекового номера группы в сигнатуре 
            lsh_dict[hash_key] = corpus[doc_id]
    
    return lsh_dict

In [85]:
def find_similar(query, lsh_dict, parameters):
    # query - document as string
    possible_duplicates = set()
    query = [query]
    shingled_query = shingling(query, parameters["k_shingles"], 
                                parameters["hash_shingles"], 
                                parameters["buckets"])  # параметры шинглирования должны быть те же, что и для корпуса
    
    query_signature = get_simhash(shingled_query[0], parameters["n_bits"])
    query_groups = get_supershingles(query_signature, n=parameters["n"])
    
    for i,group in enumerate(query_groups):
        hash_key = hash(str(group),signed=False) % (100000+i)
        if lsh_dict.get(hash_key):
            possible_duplicates.add(lsh_dict[hash_key])

    if possible_duplicates:
        print("Possible duplicates for query:")
        for doc in possible_duplicates:
            print(doc)
            
    else:
        print("Duplicates for query not founded")

In [86]:
lsh_simhash_words = lsh_training(corpus, params)

TypeError: 'signed' is an invalid keyword argument for this function

In [87]:
lsh_simhash_words

NameError: name 'lsh_simhash_words' is not defined

In [88]:
find_similar("the first king of Poad", lsh_simhash_words, params)

NameError: name 'lsh_simhash_words' is not defined

In [89]:
find_similar("the last pharaoh of Eypt", lsh_simhash_words, params)

NameError: name 'lsh_simhash_words' is not defined

# Задание:

1) Реализуйте взвешивание бинарных хешей для шинглов с помощью TF(не надо нормализовывать на длину документа). Так как для данного корпуса это бессмысленно, возьмите корпус dorn_corpus с прошлого занятия

2) Сделайте ф-ии lsh_training и find_similar универсальными, чтобы тип вычисления сигнатуры (MinHash, SimHash) можно было задавать параметром

In [90]:
dorn_corpus = ["о боже мама мама я схожу с ума", 
              "ее улыбка мама кругом голова",
              "о боже мама мама пьяный без вина", 
              "ее улыбка мама самая самая"]


# Task 2.2: LSH - Quora queries similarity

In [91]:
!pip install datasketch

    100% |████████████████████████████████| 71kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 10.3MB/s ta 0:00:01
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Воспользуемся готовой реализацией

In [92]:
import numpy as np
import pandas as pd
import re
import time
from random import randint
from datasketch import MinHash, MinHashLSHForest
import pickle

Теже самые функции шинглирования для текста

In [93]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def shingling(text, k=7, hash_shingles=False, buckets=2**15):
    text = text.lower()
    if k:
        shingled_text = [text[i:i+k] for i in range(len(text) - k+1) ]
        
    else: #if k = 0 - use words as shingles
        shingled_text = text.split()
    
    if hash_shingles: 
        shingled_text = [hash(shingle, signed=False) % buckets for shingle in shingled_text]
    
    return shingled_text


In [94]:
text = 'The devil went down to Georgia'
print('The shingles (tokens) are:', shingling(text))

The shingles (tokens) are: ['the dev', 'he devi', 'e devil', ' devil ', 'devil w', 'evil we', 'vil wen', 'il went', 'l went ', ' went d', 'went do', 'ent dow', 'nt down', 't down ', ' down t', 'down to', 'own to ', 'wn to g', 'n to ge', ' to geo', 'to geor', 'o georg', ' georgi', 'georgia']


In [95]:
def get_forest(corpus, perms):
    """
    corpus - list of docs (each doc is string)
    perms - number of permutations / of number of hash functions to use
    returns LSH forest - more effective data structure for duplicates searching
    """
    start_time = time.time()
    
    minhash = []
    
    for text in corpus:
        shingles = shingling(text)
        m = MinHash(num_perm=perms)
        for shingle in shingles:
            m.update(shingle.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)  # LSH forest - более эффективная структура для поиска дубликатов, нежели обычная хеш-таблица
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time() - start_time))
    
    return forest


In [96]:
def predict(query, corpus, forest, num_results, perms):
    """
    query - string of query for duplicates searching
    corpus - list of docs (each doc is string) - as usual
    perms - the same
    num_results - top results for searching
    forest - LSH forest structure for duplicates searching
    returns num_results docs with maximal similarity to query
    """    
    query_shingles = shingling(query)
    m = MinHash(num_perm=perms)
    for shingle in query_shingles:
        m.update(shingle.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return [] # if your query is empty, return none
    
    result = corpus[idx_array]
    
    dedupl_res = []
    for res in result:
        if (res != query) and (not res in dedupl_res):
            dedupl_res.append(res)
                
    print('\n Top Recommendations: \n')

    if len(dedupl_res):
        for i,res in enumerate(dedupl_res):
            print (i+1, res)
    else:
        print("No similar docs =(")

## Протестируем, как это работает на "Quora questions dataset"

In [97]:
df = pd.read_csv('data/quora_data/train.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [98]:
df.shape

(404290, 6)

In [99]:
texts = pd.concat([df.question1, df.question2])  # concatenate 2 columns
texts = pd.DataFrame(texts, columns=["text"])
texts = texts[pd.notnull(texts.text)]
corpus = texts.text.values  # get only texts

In [100]:
corpus

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., "What's this coin?",
       'I am having little hairfall problem but I want to use hair styling product. Which one should I prefer out of gel, wax and clay?',
       'What is it like to have sex with your cousin?'], dtype=object)

#### Построим индекс:

In [101]:
%%time
#Number of Permutations
permutations = 128

forest = get_forest(corpus, permutations)

with open("data/quora_data/forest_dump.pkl", 'wb') as f_obj:
    pickle.dump(forest, f_obj)

It took 11213.811550855637 seconds to build forest.
CPU times: user 15min 14s, sys: 24.4 s, total: 15min 38s
Wall time: 3h 7min 10s


Создание структуры занимает достаточное время. Сохраним ее и загрузим из файла уже построенный индекс, чтобы сохранить время

Файлик весит много, поэтому закину его отдельно.

In [102]:
!wget https://t.bk.ru/aZgeg/forest_dump.pkl
!mv forest_dump.pkl data/quora_data/

/bin/sh: wget: command not found
mv: forest_dump.pkl: No such file or directory


In [103]:
with open("data/quora_data/forest_dump.pkl", 'rb') as f_obj:
    forest = pickle.load(f_obj)

## Протестируем поиск похожих, взяв в качестве запроса один из случайных документов корпуса

In [104]:
permutations = 128

query = "What are the best graduate schools for studying machine learning?"
result = predict(query, corpus, forest, num_results=10, perms=permutations)



 Top Recommendations: 

1 What are the best books about machine learning?
2 What are the best books in machine learning?
3 Which are the best books for machine learning?
4 What are the most important Machine Learning algorithms?
5 What are the best books for studying machine learning?


In [105]:
for _ in range(5):
    rnd_idx = randint(0, len(corpus))
    query = corpus[rnd_idx]
    print("Query:", query)
    result = predict(query, corpus, forest, num_results=5, perms=permutations)
    print('-------------------------')



Query: How do you feel when your friend doesn't reply to your text message?

 Top Recommendations: 

1 How you feel when you graduate late than your friends?
2 Do you feel there are too many trolls on Quora?
3 How do you feel when your man notices attractive women?
-------------------------
Query: Why doesn't USA participate in Commonwealth Games?

 Top Recommendations: 

1 How and when was the Commonwealth of Nations created?
2 What is the Commonwealth of Nations?
3 What countries participate in The Commonwealth Games?
-------------------------
Query: Can you make Bisquick pancakes without eggs?

 Top Recommendations: 

1 How do you make whipped cream without heavy cream?
2 How can you make pumpkin pie without using evaporated milk?
-------------------------
Query: What is the dumbest thing any country has ever done in a war?

 Top Recommendations: 

1 What is the dumbest thing the U.S government has ever done?
2 What is the dumbest thing the U.S. government has ever done?
3 What's th